In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 產品說明產生器來自影像

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_image.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google 協作平台標誌"><br>在協作平台運行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_image.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/description-generation/product_description_generator_image.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者| [Lavi Nigam](https://github.com/lavinigam-gcp) |


## 總覽

此筆記本顯示如何根據圖片建立產品說明之使用案例。在此筆記本中，你會使用 [服裝圖片資料集](https://github.com/alexeygrigorev/clothing-dataset-small) 為服飾圖片建立產品說明。
作為初始步驟，你會在 Vertex AI 中部署預先訓練好的 [BLIP 圖片字幕](https://huggingface.co/Salesforce/blip-image-captioning-base) 模型，以進行線上預測。然後你將使用模型加上圖片字幕。稍後，你會使用產品圖片的字幕，搭配 PaLM 模型產生行銷附件的說明。

### 目標

- 將模型上傳至 [模型登錄管理中心](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 在 [端點](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints) 上部署模型。
- 執行圖片字幕的線上預測。
- 搭配圖片字幕執行 PaLM 模型的線上預測，以產生產品說明。

### 費用

本教學課程使用 Google Cloud 的計費元件：

* Vertex AI
* Cloud Storage

深入瞭解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定價](https://cloud.google.com/storage/pricing)，並使用 [定價計算器](https://cloud.google.com/products/calculator/) 根據預測使用量產生成本估算。


## 開始使用


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --quiet --user

***Colab 獨有** *：取消以下Cell註解以重新啟動核心，或使用按鈕重新啟動核心。對於 Vertex AI Workbench，可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

### 匯入函式庫


**Colab 專用：** 取消下一個Cell註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，你不需要執行這項作業。


In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import io
import os
import base64
import requests
from PIL import Image
from io import BytesIO
from datetime import datetime
from google.cloud import storage
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel

### 載入模型


In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### 下列 GCS Bucket 包含一些時尚產品圖像資料集範例


In [ ]:
GCS_BUCKET = "github-repo"
!gsutil ls gs://$GCS_BUCKET/product_img/

### 定義常數


In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/pytorch-transformers-serve"

### 定義常見功能


In [ ]:
def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(model_id, task):
    model_name = "blip-image-captioning"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
        deploy_request_timeout=1800,
    )
    return model, endpoint


def read_jpeg_image_from_gcs(bucket_name, image_name):
    """Reads a JPEG image from a Google Cloud Storage (GCS) bucket.

    Args:
    bucket_name: The name of the GCS bucket that contains the image file.
    image_name: The name of the image file in the GCS bucket.

    Returns:
    The image file as a PIL Image object.
    """

    # Import the Google Cloud Storage client library.

    # Create a storage client.
    client = storage.Client()

    # Get the bucket object.
    bucket = client.bucket(bucket_name)

    # Get the blob object.
    blob = bucket.blob(image_name)

    # Read the blob to a bytestring.
    image_data = blob.download_as_bytes()

    # Decode the bytestring to a PIL Image object.
    image = Image.open(io.BytesIO(image_data))

    # Return the PIL Image object.
    return image

## 上傳並部署模型


此部分會將預訓練模型上傳到 Model Registry，並於端點中以 1 T4 GPU 部署。

模型部署步驟將花費大約 15 分鐘。

部署後，你可以傳送圖片以取得說明。


**備註：** **只執行這個Cell一次。** 由於這是私有端點，因此，最多只能將一個模型配署至每個私有端點。


In [ ]:
model, endpoint = deploy_model(
    model_id="Salesforce/blip-image-captioning-base", task="image-to-text"
)

現在你將撰寫一個函式，它會將 BLIP 模型產生的圖片標題傳送給我們的 PaLM 2 文字產生模型。透過你的提示，你希望它回傳可在行銷材料中使用的朗朗上口產品說明。


In [ ]:
def generate_product_description(model, image_caption, temperature=0):
    """Ideation example with a Large Language Model"""
    prompt_prefix = "Imagine you are a digital marketer working for a retail organization. \
                    You are an expert in building detailed and catchy descriptions fro the retail fashion products on your website.\
                    Generate a product description using the following short caption that describes the apparel"
    prompt = prompt_prefix + image_caption
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=256,
        top_k=40,
        top_p=0.8,
    )
    return response.text

一旦定義模型，請測試圖片及其標題，以了解如何產生說明。你可以撰寫一個簡單的 for 迴圈，遍歷原始資料夾中的每張照片，並透過傳遞標題來呼叫我們的模型。你可以閱讀每張圖片的說明。


In [ ]:
from matplotlib import pyplot as plt

for i in range(1, 9):
    image_data = read_jpeg_image_from_gcs(
        GCS_BUCKET, "product_img/fashion" + str(i) + ".jpeg"
    )
    # Display the image
    plt.imshow(image_data)
    plt.axis("off")
    plt.show()

    instances = [
        {"image": image_to_base64(image_data)},
    ]
    preds = endpoint.predict(instances=instances).predictions
    print(preds)
    product_description = generate_product_description(
        model=generation_model, image_caption=preds[0]
    )
    print(product_description)